In [ ]:
from sent_bert_emb_viz_util import load_data, encode_data_tuple, get_model_from_encoding_output, test_model_from_encoding_output
import random
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier

import pickle

from step4_further_results_from_annotations import get_and_display_results,rule_based_model_ensemble
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import pandas as pd

In [ ]:
# # input the encoding output_tuple
# # output the trained model
# # test_size portion \in (0,1) default as 0.1, not outputting validation results if test_size as 0.
# # balanced sampling default as False
# # regularisation penalty C default as 1
# def get_model_from_encoding_output(output_tuple, num_of_training_samples, test_size=0.1, balanced_sampling=False, C=1):
#     if len(output_tuple) == 2:
#         X, y = output_tuple
#         list_ind_empty_cw, list_ind_wrong_find_mt = [], []
#     else:
#         X, y, list_ind_empty_cw, list_ind_wrong_find_mt = output_tuple
#     # X, y = load_data('mention_disamb_data_features_NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12_sample_89902.pik')
#     #print('number of entries with issues: %s+%s' % str(len(list_ind_empty_cw)), str(len(list_ind_wrong_find_mt)))
#     #print(list_ind_wrong_find_mt)
#     # if not masked_training:
#         # #remove the list of index there were eliminated during the wrong_find_mention_token process
#         # list_ind_wrong_find_mt = [26964,41622,51767,65705,67399,85607,87693]
#         # y = np.array([ele for ind, ele in enumerate(y.tolist()) if ind not in list_ind_wrong_find_mt])
    
#     print('encoding done')
#     #print(y.shape, type(y), len(y.tolist()))
#     print('X,y:', X.shape, type(X), y.shape, type(y))
#     unique, counts = np.unique(y, return_counts=True)
#     #print(unique, counts)
#     print('whole data pos:', counts[1], 'neg:', counts[0])

#     #data standardisation
#     #X = preprocessing.scale(X)

#     #data split
#     if test_size != 0:
#         X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=test_size, random_state=1234)
#     else:
#         # if test size not as 0, then do not do data split
#         X_train, y_train = X, y
        
#     training_data_len = X_train.shape[0]
#     print(training_data_len)
    
#     if not balanced_sampling:
#         #adjust the number of training samples by the error samples
#         #this ensure the different training settings will use the exact same set of training data (while adjusted numbers may be slightly different)
#         num_of_training_samples_adjusted = num_of_training_samples
#         for ind in list_ind_empty_cw + list_ind_wrong_find_mt:
#             if ind < num_of_training_samples:
#                 num_of_training_samples_adjusted = num_of_training_samples_adjusted - 1            
#         #try only a random sample of the whole training data: the first num_of_training_samples (adjusted by the error samples), X_train is the randomly shuffled data.
#         X_train = X_train[:num_of_training_samples_adjusted,:]
#         y_train = y_train[:num_of_training_samples_adjusted]
#         print(X_train.shape, type(X_train), type(y_train))
#     else:
#         #sample a label-balanced set from X_train and Y_train
#         #and the number of the whole set is equal to num_of_training_samples
#         num_tr_samples_per_label = round(num_of_training_samples/2)
#         #print(type(num_tr_samples_per_label), num_tr_samples_per_label)
#         data_ind_neg = np.where(y_train == 0)[0]
#         data_ind_pos = np.where(y_train == 1)[0]
#         data_ind_balanced_sample = np.sort(np.concatenate([data_ind_pos[:num_tr_samples_per_label], data_ind_neg[:num_tr_samples_per_label]]))
#         X_train = X_train[data_ind_balanced_sample]
#         y_train = y_train[data_ind_balanced_sample]
#         print(X_train.shape, type(X_train), type(y_train))
    
#     _, counts = np.unique(y_train, return_counts=True)
#     print('training data pos:', counts[1], 'neg:', counts[0])
    
#     #train
#     #clf = SVC(kernel="linear", C=0.025)
#     clf = LogisticRegression(C=C, penalty='l2') #solver='liblinear' #max_iter=500
#     #clf = LinearRegression()
#     #clf = MLPClassifier(hidden_layer_sizes=10)
#     clf.fit(X_train, y_train)
#     #get training results
#     y_pred_train = clf.predict(X_train)
#     print('training precision:', precision_score(y_pred_train, y_train), 'recall:', recall_score(y_pred_train, y_train), 'F1:', f1_score(y_pred_train, y_train))

#     if test_size != 0:
#         #get validation results
#         y_pred = clf.predict(X_valid)
#         print(y_pred, type(y_pred))
#         #y_pred = y_pred.round() # for linear regression
#         #print(y_pred, y_valid)
#         print('validation precision:', precision_score(y_pred, y_valid), 'recall:', recall_score(y_pred, y_valid), 'F1:', f1_score(y_pred, y_valid))
    
#     return clf

In [ ]:
# def test_model_from_encoding_output(output_tuple_test, num_of_testing_samples, clf):    
#     if len(output_tuple_test) == 2:
#         X_test, y_test = output_tuple_test
#         list_ind_empty_cw_test, list_ind_wrong_find_mt_test = [], []
#     else:
#         # it also outputs the problematic data entry indexes. we will use these when we fill the results into the spreadsheet.
#         X_test, y_test, list_ind_empty_cw_test, list_ind_wrong_find_mt_test = output_tuple_test          
#     print('list_ind_empty_cw_test:', list_ind_empty_cw_test)
#     print('list_ind_wrong_find_mt_test:', list_ind_wrong_find_mt_test)
    
#     list_ind_err_test = list_ind_empty_cw_test + list_ind_wrong_find_mt_test
#     list_ind_err_test.sort() # sort the index list from low to high
#     #adjust the number of testing samples by the error samples
#     num_of_testing_samples_adjusted = num_of_testing_samples
#     for ind in list_ind_empty_cw_test + list_ind_wrong_find_mt_test:
#         if ind < num_of_testing_samples:
#             num_of_testing_samples_adjusted = num_of_testing_samples_adjusted - 1
            
#     print('num_of_whole_testing_data:', len(y_test))
#     # to note: there might be unlabelled data in y_test, otherwise 0 or 1.
#     # get the labelled part in y_test
#     y_test_labelled = [y_test[ind] for ind, y_test_ele in enumerate(y_test) if (y_test_ele == 0 or y_test_ele == 1) and ind<num_of_testing_samples_adjusted]
#     print('num_of_actual_testing_data:',len(y_test_labelled))
    
#     # check the filtered out testing samples
#     for ind, y_test_ele in enumerate(y_test):
#         if not ((y_test_ele == 0 or y_test_ele == 1)) and ind<num_of_testing_samples_adjusted:
#             print('not labelled test sample:',ind,y_test_ele)
#             #list_ind_err_test.append(ind) - this is not the original ind, to fix later
#     #data standardisation
#     #X_test = preprocessing.scale(X_test)

#     y_pred_test = clf.predict(X_test)
#     #y_pred_test = y_pred_test.round() # for linear regression
    
#     #print('model results all %s:' % str(len(y_test_labelled)))
#     y_pred_test_labelled = [y_pred_test[ind] for ind, y_test_ele in enumerate(y_test) if  (y_test_ele == 0 or y_test_ele == 1) and ind<num_of_testing_samples_adjusted]
        
#     return y_test_labelled, y_pred_test_labelled, list_ind_err_test

In [ ]:
fill_data = True
calculate_baseline_results = False
test_with_non_rule_annotated_only = False
#num_of_testing_items = 50 # this applies only when test_with_non_annotated_only is False

masked_training = False
use_doc_struc = True

rule_based_ensembling = True
data_ensembling = False
mj_rate = 0.15 #1.0
num_sample = 100 #len(data_list_tuples) #500 #len(data_list_tuples) #1000 for quick checking of the program

# data selection approaches: random sampling, balanced random sampling, and diverse sampling
num_of_training_samples_non_masked = 9000 # much more than the whole data, i.e. using the whole data # 9000
num_of_training_samples_masked = 500 # much more than the whole data, i.e. using the whole data # 500
balanced_random_sampling=False
diverse_sampling=False

num_of_data_per_mention=25
masking_rate = 1 #0.15 # to implement
window_size = 5
C = 1 # l2 regularisation parameter

num_of_testing_samples = 400

#model_path='bert-models\\pubmedBERT_tf_model\\'; model_name = 'pubmedBERT'
#model_path='bert-models\\NCBI_BERT_pubmed_mimic_uncased_L-12_H-768_A-12\\'; model_name = 'blueBERTnorm'
#model_path='bert-models\\NCBI_BERT_pubmed_mimic_uncased_L-24_H-1024_A-16\\'; model_name = 'blueBERTlarge'

marking_str_tr = 'training'

#marking_str_te = 'testing_200'    
#marking_str_te = 'testing_non_rule_anno'
marking_str_te = 'testing_1073'

# if masked_training:
#     masking_str = '_[MASK]' if masking_rate == 1 else '_[MASK]%s' % str(masking_rate)
# else:
#     masking_str = ''
        
trained_models_name = 'model_%s_ws%s%s%s%s.pik' % (model_name, str(window_size), '_ds' if use_doc_struc else '', '_divs%s' % str(num_of_data_per_mention) if diverse_sampling else '','_nm%sm%s' % (str(num_of_training_samples_non_masked),str(num_of_training_samples_masked)))
#print(trained_models_name)

In [ ]:
#1. load data, encoding, and train model 
#load data
data_list_tuples = load_data('mention_disamb_data.pik')
random.Random(1234).shuffle(data_list_tuples) #randomly shuffle the list with a random seed        
#data_list_tuples = data_list_tuples[0:num_sample] # set a small sample for quick testing
print(len(data_list_tuples))

print('start encoding')

#encoding
output_tuple_masked = encode_data_tuple(data_list_tuples, masking=True, with_doc_struc=False, model_path=model_path, marking_str=marking_str_tr, window_size=window_size, masking_rate=masking_rate, diverse_sampling=diverse_sampling, num_of_data_per_mention=num_of_data_per_mention)
output_tuple_non_masked_ds = encode_data_tuple(data_list_tuples, masking=False, with_doc_struc=True, model_path=model_path, marking_str=marking_str_tr, window_size=window_size, masking_rate=masking_rate, diverse_sampling=diverse_sampling, num_of_data_per_mention=num_of_data_per_mention)

#training
clf_model_masked = get_model_from_encoding_output(output_tuple_masked,num_of_training_samples_masked,balanced_sampling=balanced_random_sampling)
clf_model_non_masked_ds = get_model_from_encoding_output(output_tuple_non_masked_ds,num_of_training_samples_non_masked,balanced_sampling=balanced_random_sampling)

#export models
with open(trained_models_name, 'ab') as data_f:
    pickle.dump((clf_model_non_masked_ds,clf_model_masked), data_f)
    print('\n' + trained_models_name, 'saved')

In [ ]:
#2. load testing data and predict results: 
#load data from .xlsx and save the results to a specific column
# get a list of data tuples from an annotated .xlsx file
# data format: a 6-element tuple of section_text, document_structure_name, mention_name, UMLS_code, UMLS_desc, label (True or False)
df = pd.read_excel('for validation - SemEHR ori.xlsx')
# change nan values into empty strings in the two rule-based label columns
df[['neg label: only when both rule 0','pos label: both rules applied']] = df[['neg label: only when both rule 0','pos label: both rules applied']].fillna('') 
# if the data is not labelled, i.e. nan, label it as -1 (not positive or negative)
df[['manual label from Hang']] = df[['manual label from Hang']].fillna(-1) 

data_list_tuples = []
for i, row in df.iterrows():
    if test_with_non_rule_annotated_only: # only test with those not annotated by rules
        #print(row['neg label: only when both rule 0'], type(row['neg label: only when both rule 0']))
        if row['neg label: only when both rule 0'] != '' or row['pos label: both rules applied'] != '':
            continue
    doc_struc = row['document structure']
    text = row['Text']
    mention = row['mention']
    UMLS_code = row['UMLS with desc'].split()[0]
    UMLS_desc = ' '.join(row['UMLS with desc'].split()[1:])
    #label = row['manual label from Hang']
    label = row['gold text-to-UMLS label']
    label = 0 if label == -1 else label # assume that the inapplicable (-1) entries are all False.
    #print(label)
    data_tuple = (text,doc_struc,mention,UMLS_code,UMLS_desc,label)
    #if i<2:
    #    print(data_tuple)
    data_list_tuples.append(data_tuple)

In [ ]:
# get testing data rep and predict with the model
# encoding
output_tuple_test_masked = encode_data_tuple(data_list_tuples, masking=True, with_doc_struc=False, model_path=model_path, marking_str=marking_str_te, window_size=window_size, masking_rate=masking_rate)
output_tuple_test_non_masked_ds = encode_data_tuple(data_list_tuples, masking=False, with_doc_struc=True, model_path=model_path, marking_str=marking_str_te, window_size=window_size, masking_rate=masking_rate)

# prediction
print('single model results')
print('masked training:')
y_test_labelled_masked, y_pred_test_labelled_masked,_ = test_model_from_encoding_output(output_tuple_test_masked, num_of_testing_samples, clf_model_masked)
get_and_display_results(y_test_labelled_masked, y_pred_test_labelled_masked)

print('non-masked training with ds:')
y_test_labelled_non_masked_ds, y_pred_test_labelled_non_masked_ds,list_of_err_samples_non_masked = test_model_from_encoding_output(output_tuple_test_non_masked_ds, num_of_testing_samples, clf_model_non_masked_ds)
get_and_display_results(y_test_labelled_non_masked_ds, y_pred_test_labelled_non_masked_ds)
#also returned the list of erroneous samples using the non-masked encoding.

In [ ]:
#rule-based ensembling
y_pred_ment_len = df[['rule (mention length >3)']].to_numpy()
y_pred_prevalence = df[['rule (prevalance th <= 0.005)']].to_numpy()

y_pred_ment_len_labelled = np.array([y_pred_ment_len[ind] for ind in range(len(y_pred_ment_len)) if ind<num_of_testing_samples and ind not in list_of_err_samples_non_masked])
print('y_pred_ment_len_labelled:',len(y_pred_ment_len_labelled))
# here we ignored the possibility that if some testing data not labelled to 0 or 1.

y_pred_prevalence_labelled = np.array([y_pred_prevalence[ind] for ind in range(len(y_pred_prevalence)) if ind<num_of_testing_samples and ind not in list_of_err_samples_non_masked])
print('y_pred_prevalence_labelled:',len(y_pred_prevalence_labelled))

print('rule-based model ensemble best scenario results:')
# #y_pred_test_m_labelled_ensemb = np.logical_or(y_pred_test_m_labelled,y_pred_test_m_ds_large_labelled).astype(int)
y_pred_rule_based_model_ensemble = rule_based_model_ensemble(y_pred_ment_len_labelled, y_pred_prevalence_labelled, y_pred_test_labelled_non_masked_ds, y_pred_test_labelled_masked)
get_and_display_results(y_test_labelled_non_masked_ds, y_pred_rule_based_model_ensemble)